In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# Coordinates for tornado disasters in Columbus, 1950-2000: http://www.usa.com/columbus-oh-natural-disasters-extremes.htm
# df = pd.read_csv()
import requests
import bs4 as bs
import re
import numpy as np
import pandas as pd
from collections import Counter

src = requests.get("http://www.usa.com/columbus-oh-natural-disasters-extremes.htm")
soup = bs.BeautifulSoup(src.content, features='html.parser')

print(soup.text)



 Columbus, OH Natural Disasters and Weather Extremes - USA.com™


 




Local Data Search  tyhInit();
 


USA.com / Ohio / Columbus, OH / Natural Disasters and Weather Extremes
Columbus, OH Natural Disasters and Weather ExtremesBasic InfoPopulation/RacesIncome/CareersHousingEducationCrime/Crime RateOthersSchool District | Public Schools | Private Schools | Public Library Weather | Natural Extremes | Air Quality | Environmental Watch | Zip Codes | Government  Hot RankingsFastest / Slowest Growing Cities NearbyBest / Worst Cities by Crime Rate NearbyRichest / Poorest Cities by Income NearbyExpensive / Cheapest Homes NearbyMost / Least Educated Cities NearbyFastest / Slowest Growing Cities in OHHigh / Low OH Cities by Males EmployedHigh / Low OH Cities by Females EmployedBest / Worst Cities by Crime Rate in OHRichest / Poorest Cities by Income in OHExpensive / Cheapest Homes by City in OHMost / Least Educated Cities in OHThe chance of earthquake damage in Columbus is about the same as O